In [ ]:
print("Running")

In [ ]:
import sys
import gym
import glob
import torch
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../")
from ACDir import ActorCriticDiscrete

In [ ]:
deg2rad = lambda x: x * 3.1415926536 / 180.
rad2deg = lambda x: x * 180. / 3.1415926536

In [ ]:
attempt = 9
num_actions = 2
ac = True
rl_type = "acdd" if ac else "mfmaac"

In [ ]:
paths = sorted(glob.glob(f"weights/{rl_type}{attempt:03d}*.pt"))
path = paths[7]
paths

In [ ]:
model = ActorCriticDiscrete(num_inputs=1, num_actions=num_actions)
model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))#"../weights/mfmaac_epochs800_measure500_timesteps10_20210216-213727.pt"))
model.eval()

In [ ]:
actions = []
action_meanings = torch.tensor([-1, 0, 1]) if num_actions == 3 else torch.tensor([-1, 1])
ran = list(range(-90,270))
plt.rcParams.update({'font.size': 22})
for Delta in ran[90:90+180]:
    Delta_orig = Delta
    Delta = torch.tensor(deg2rad(Delta)).reshape((1,)).float()
    Delta = torch.nn.functional.relu(model.affine(Delta))
    #print("model.action_layer(Delta)", model.action_layer(Delta).size())
    action_probs = torch.nn.functional.softmax(model.action_layer(Delta), dim=0)    
    #print()
    actions.append((action_probs * action_meanings * num_actions).mean().detach())

fig = plt.figure(figsize=(20,10))
plt_line, = plt.plot(ran[90:90+180], actions, c="black", label="Avg. action $\Delta_{t+1}$")
#plt.title(f"$\Delta_{{opt}}$ = {(torch.tensor(actions)**2).argmin()}°")
plt.xlabel("$\Delta$ °")
plt.ylabel("Avg action $\Delta_{t+1}$");
actual_optimum = plt.axvline(x=76, ls="--", c="red", label=f"Expected optimum $\Delta_{{opt}}$ = 76°")
found_optimum = plt.axvline(x=(torch.tensor(actions)**2).argmin(), ls="--", c="blue", label=f"Found optimum $\Delta$ = {(torch.tensor(actions)**2).argmin()}°")
plt.legend(handles=[plt_line, actual_optimum, found_optimum])
#fig.savefig('avg_action_ac.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
values = []
ran = list(range(-90,270))
for Delta in ran:
    Delta = torch.tensor(deg2rad(Delta)).reshape((1,)).float()
    Delta = torch.nn.functional.relu(model.affine(Delta))
    state_value = model.value_layer(Delta)
    values.append(state_value)

plt.plot(ran, values)
#env.load(glob.glob("../runs/mfmaac*")[-1])

In [ ]:
for Delta in ran[90:90+180:15]:
    continue
    Delta_orig = Delta
    Delta = torch.tensor(deg2rad(Delta)).reshape((1,)).float()
    Delta = torch.nn.functional.relu(model.affine(Delta))
    action_probs = torch.nn.functional.softmax(model.action_layer(Delta))    
    #print([-1, 0, 1], action_probs)
    fig, ax = plt.subplots()
    ax.set_title(f'Delta = {Delta_orig}°')
    ax.bar([-1, 0, 1], action_probs.detach().numpy(), )
    plt.show()

In [ ]:
actions = []
action_meanings = torch.tensor([-1, 0, 1]) if num_actions == 3 else torch.tensor([-1, 1])
for Delta in ran[89:90+180]:
    Delta_orig = Delta
    Delta = torch.tensor(deg2rad(Delta)).reshape((1,)).float()
    Delta = torch.nn.functional.relu(model.affine(Delta))
    action_probs = torch.nn.functional.softmax(model.action_layer(Delta))    
    #print()
    actions.append((action_probs * action_meanings * num_actions).mean().detach())

actions = torch.tensor(actions)
diff_actions = actions[1:]-actions[:-1]
print(diff_actions.argmin())
plt.plot(ran[90:90+180], diff_actions)
plt.scatter(diff_actions.argmin(), diff_actions.min(), c="r")
plt.xlabel("$\Delta$ °")
plt.ylabel("Avg action $\Delta_{t+1}$");

In [ ]:
pd.read_csv(f"runs/loss{attempt:03d}.csv").plot(x="epoch", y="loss")